# Problem Definition

This notebook focus on creating a successful machine learning that predicts coffee bean origin country.

# Data Acquisition Libraries and Datasets

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import string
import random

In [3]:
# import/convert csv to pd
train_url = "./data/train.csv"
test_url = "./data/test.csv"

train_df = pd.read_csv(train_url).rename(columns={"Unnamed: 0": "id"})
test_df = pd.read_csv(test_url).drop(columns={"Unnamed: 0": "id"})

print("TRAIN DATA SHAPE: ", train_df.shape)
print("TEST DATA SHAPE: ", test_df.shape)

TRAIN DATA SHAPE:  (585, 29)
TEST DATA SHAPE:  (147, 27)


In [4]:
train_df[train_df.notna()]

,id,Species,Farm.Name,Lot.Number,Altitude,Number.of.Bags,Bag.Weight,Harvest.Year,Grading.Date,Variety,...,Sweetness,Cupper.Points,Total.Cup.Points,Moisture,Category.One.Defects,Quakers,Color,Category.Two.Defects,Expiration,Country.of.Origin
0,0,Arabica,conquista / morito,NaN,NaN,250,1 kg,2012,"January 13th, 2012",Bourbon,...,10.0,7.00,78.33,0.11,0,0.0,Green,9,"January 12th, 2013",2
1,1,Arabica,NaN,NaN,de 1.600 a 1.950 msn,275,70 kg,Mayo a Julio,"July 29th, 2011",Caturra,...,10.0,8.17,83.08,0.01,0,0.0,NaN,0,"July 28th, 2012",1
2,2,Arabica,la esmeralda,11/23/0634,4000,25,69 kg,2017,"September 8th, 2017",Bourbon,...,10.0,7.50,82.58,0.10,0,1.0,Green,2,"September 8th, 2018",2
3,3,Arabica,fazenda santo antonio,NaN,900-1100,305,2 kg,2014,"February 13th, 2015",Catuai,...,10.0,7.58,83.00,0.00,0,0.0,Green,0,"February 13th, 2016",0
4,4,Arabica,NaN,NaN,NaN,1,5 lbs,2013,"September 12th, 2014",NaN,...,10.0,7.25,82.08,0.11,1,0.0,Green,0,"September 12th, 2015",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,580,Arabica,NaN,NaN,1800,275,2 kg,2015/2016,"June 5th, 2015",Other,...,10.0,7.83,84.25,0.12,0,0.0,Green,0,"June 4th, 2016",1
581,581,Arabica,cafetal,101,1300,200,69 kg,2017,"July 3rd, 2017",Mundo Novo,...,10.0,7.83,83.92,0.11,0,0.0,Green,5,"July 3rd, 2018",3
582,582,Arabica,las lomas,NaN,1200,250,1 kg,2012,"July 11th, 2012",Bourbon,...,10.0,7.67,83.00,0.11,5,0.0,Green,8,"July 11th, 2013",3
583,583,Arabica,NaN,NaN,1800 msnm,250,70 kg,4T/10,"February 9th, 2011",NaN,...,10.0,7.42,82.17,0.08,0,0.0,NaN,0,"February 9th, 2012",1


# Data Exploration: Familiarize The Data

In [5]:
train_df

,id,Species,Farm.Name,Lot.Number,Altitude,Number.of.Bags,Bag.Weight,Harvest.Year,Grading.Date,Variety,...,Sweetness,Cupper.Points,Total.Cup.Points,Moisture,Category.One.Defects,Quakers,Color,Category.Two.Defects,Expiration,Country.of.Origin
0,0,Arabica,conquista / morito,NaN,NaN,250,1 kg,2012,"January 13th, 2012",Bourbon,...,10.0,7.00,78.33,0.11,0,0.0,Green,9,"January 12th, 2013",2
1,1,Arabica,NaN,NaN,de 1.600 a 1.950 msn,275,70 kg,Mayo a Julio,"July 29th, 2011",Caturra,...,10.0,8.17,83.08,0.01,0,0.0,NaN,0,"July 28th, 2012",1
2,2,Arabica,la esmeralda,11/23/0634,4000,25,69 kg,2017,"September 8th, 2017",Bourbon,...,10.0,7.50,82.58,0.10,0,1.0,Green,2,"September 8th, 2018",2
3,3,Arabica,fazenda santo antonio,NaN,900-1100,305,2 kg,2014,"February 13th, 2015",Catuai,...,10.0,7.58,83.00,0.00,0,0.0,Green,0,"February 13th, 2016",0
4,4,Arabica,NaN,NaN,NaN,1,5 lbs,2013,"September 12th, 2014",NaN,...,10.0,7.25,82.08,0.11,1,0.0,Green,0,"September 12th, 2015",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,580,Arabica,NaN,NaN,1800,275,2 kg,2015/2016,"June 5th, 2015",Other,...,10.0,7.83,84.25,0.12,0,0.0,Green,0,"June 4th, 2016",1
581,581,Arabica,cafetal,101,1300,200,69 kg,2017,"July 3rd, 2017",Mundo Novo,...,10.0,7.83,83.92,0.11,0,0.0,Green,5,"July 3rd, 2018",3
582,582,Arabica,las lomas,NaN,1200,250,1 kg,2012,"July 11th, 2012",Bourbon,...,10.0,7.67,83.00,0.11,5,0.0,Green,8,"July 11th, 2013",3
583,583,Arabica,NaN,NaN,1800 msnm,250,70 kg,4T/10,"February 9th, 2011",NaN,...,10.0,7.42,82.17,0.08,0,0.0,NaN,0,"February 9th, 2012",1


In [6]:
# check features values summary
col_values = {}
for col in train_df.columns[:]:
    col_values[col] = train_df[col].unique().tolist()
print(f"FEATURES VALUES SUMMARY\n-----")
for col in col_values:
    print(f"{col}: {col_values[col]}")

FEATURES VALUES SUMMARY
-----
id: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 

In [7]:
# check for the dataset features
output_str = ""
num = 1
for col in col_values: 
    jump_line = (f"\n" if num % 8 == 0 else "")
    output_str += col + ", " + jump_line
    num += 1
print(f"FEATURES SUMMARY\n-----")
print(output_str)

FEATURES SUMMARY
-----
id, Species, Farm.Name, Lot.Number, Altitude, Number.of.Bags, Bag.Weight, Harvest.Year, 
Grading.Date, Variety, Processing.Method, Aroma, Flavor, Aftertaste, Acidity, Body, 
Balance, Uniformity, Clean.Cup, Sweetness, Cupper.Points, Total.Cup.Points, Moisture, Category.One.Defects, 
Quakers, Color, Category.Two.Defects, Expiration, Country.of.Origin, 


In [8]:
# create insightful features category
category = {
    "location": ["Farm.Name", "Country.of.Origin", "Altitude"],
    "logistic": ["Lot.Number", "Number.of.Bags", "Bag.Weight", "Harvest.Year", "Grading.Date", "Expiration"],
    "plant": ["Species", "Variety", "Processing.Method"],
    "taste": ["Aroma", "Flavor", "Aftertaste", "Acidity", "Body", "Balance", "Sweetness", "Uniformity", "Clean.Cup"],
    "quality": ["Cupper.Points", "Total.Cup.Points"],
    "physical": ["Moisture", "Category.One.Defects", "Category.Two.Defects", "Quakers", "Color"]
}
print(f"FEATURES CATEGORIZE SUMMARY\n-----")
for cat in category:
    print(cat)
    print(category[cat])
    print("\n")

FEATURES CATEGORIZE SUMMARY
-----
location
['Farm.Name', 'Country.of.Origin', 'Altitude']


logistic
['Lot.Number', 'Number.of.Bags', 'Bag.Weight', 'Harvest.Year', 'Grading.Date', 'Expiration']


plant
['Species', 'Variety', 'Processing.Method']


taste
['Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance', 'Sweetness', 'Uniformity', 'Clean.Cup']


quality
['Cupper.Points', 'Total.Cup.Points']


physical
['Moisture', 'Category.One.Defects', 'Category.Two.Defects', 'Quakers', 'Color']




In [9]:
train_df.isna().sum()

id                        0
Species                   0
Farm.Name               166
Lot.Number              480
Altitude                 67
Number.of.Bags            0
Bag.Weight                0
Harvest.Year             10
Grading.Date              0
Variety                  61
Processing.Method        52
Aroma                     0
Flavor                    0
Aftertaste                0
Acidity                   0
Body                      0
Balance                   0
Uniformity                0
Clean.Cup                 0
Sweetness                 0
Cupper.Points             0
Total.Cup.Points          0
Moisture                  0
Category.One.Defects      0
Quakers                   1
Color                   116
Category.Two.Defects      0
Expiration                0
Country.of.Origin         0
dtype: int64

In [10]:
test_df.isna().sum()

Species                   0
Farm.Name                49
Lot.Number              118
Altitude                 20
Number.of.Bags            0
Bag.Weight                0
Harvest.Year              2
Grading.Date              0
Variety                  12
Processing.Method        12
Aroma                     0
Flavor                    0
Aftertaste                0
Acidity                   0
Body                      0
Balance                   0
Uniformity                0
Clean.Cup                 0
Sweetness                 0
Cupper.Points             0
Total.Cup.Points          0
Moisture                  0
Category.One.Defects      0
Quakers                   0
Color                    26
Category.Two.Defects      0
Expiration                0
dtype: int64

In [11]:
# check for nan values on both datasets
print(f"TRAIN DATASET NAN VALUES COLS: \n-----\n", train_df.columns[train_df.isna().any()].tolist())
print("\n")
print(f"TEST DATASET NAN VALUES COLS: \n-----\n", test_df.columns[test_df.isna().any()].tolist())

TRAIN DATASET NAN VALUES COLS: 
-----
 ['Farm.Name', 'Lot.Number', 'Altitude', 'Harvest.Year', 'Variety', 'Processing.Method', 'Quakers', 'Color']


TEST DATASET NAN VALUES COLS: 
-----
 ['Farm.Name', 'Lot.Number', 'Altitude', 'Harvest.Year', 'Variety', 'Processing.Method', 'Color']


# Data Preparation: Data Consistency

In [12]:
# align date datatype
numerical_cols = ["Altitude", "Number.of.Bags", "Bag.Weight", "Aroma", 
                 "Flavor", "Aftertaste", "Acidity", "Body", "Balance", 
                 "Uniformity", "Clean.Cup", "Sweetness", "Cupper.Points",
                 "Total.Cup.Points", "Moisture", "Category.One.Defects", 
                 "Quakers", "Category.Two.Defects"
                 ]
date_cols = ["Harvest.Year", "Grading.Date", "Expiration"]

def processNumericalRow(row, col):
    word = str(row)
    numbers = [
        float((number.replace(".", "") if col == "Altitude" else number)) 
        for number in re.findall("\d+\.\d+|\d+", word)
        ]
    if len(numbers) > 1:
        mean = sum(numbers) / len(numbers)
        return mean
    elif len(numbers) == 1:
        first_number = numbers[0]
        return first_number
    else:
        return None
    
def processDateRow(row, col):
    word = str(row)
    if col == "Harvest.Year":
        # 2012
        if re.fullmatch("\d{4}", word):
            return word + "/" + word
        # 4t/2012 or 4T/2012
        elif re.fullmatch("[1-4][Tt]/\d{4}", word):
            _, year = re.split("[Tt]/", word)
            return year + "/" + year 
        # 4t/12
        elif re.fullmatch("[1-4][Tt]/\d{2}", word):
            _, year = re.split("[Tt]/", word)
            return "20" + year + "/" + "20" +year
        # 2012 / 2012
        elif re.fullmatch("\d{4} / \d{4}", word):
            year1, year2 = re.split(" / ", word)
            return year1 + "/" + year2
        # 2012-2012
        elif re.fullmatch("\d{4}-\d{4}", word):
            year1, year2 = re.split("-", word)
            return year1 + "/" + year2
        # .../2012
        elif re.fullmatch(".*(Julio /\d{4}|47/\d{4})$", word):
            _, year = word.split("/")
            return year + "/" + year
        # 4T72010
        elif word == "4T72010":
            return "2010" + "/" + "2010"
        # 08/09 crop
        elif word == "Jan.11":
            return "2011" + "/" + "2011"
        # 08/09 crop
        elif word == "08/09 crop":
            return "2008" + "/" + "2009"
        # 2012/2012 or others
        else:
            if re.fullmatch("\d{4}/\d{4}", word):
                return word
            else:
                list_words = word.split(" ")
                combined_word = None
                for word_list in list_words:
                    if re.fullmatch("\d{4}", word_list):
                        combined_word = word_list + "/" + word_list
                return combined_word
    else:
        return pd.Timestamp(row).date()
            
def processCol(df):
    for col in df.columns:
        if col in numerical_cols:
            df[col] = df[col].apply(lambda row: None if pd.isna(row) else processNumericalRow(row, col))
        elif col in date_cols:
            df[col] = df[col].apply(lambda row: None if pd.isna(row) else processDateRow(row, col))
        else:
            continue
        
processCol(train_df)
processCol(test_df)

# list_of_years = [row for row in train_df["Grading.Date"] if isinstance(row, str)]
# print(list_of_years) 

train_df.head()

,id,Species,Farm.Name,Lot.Number,Altitude,Number.of.Bags,Bag.Weight,Harvest.Year,Grading.Date,Variety,...,Sweetness,Cupper.Points,Total.Cup.Points,Moisture,Category.One.Defects,Quakers,Color,Category.Two.Defects,Expiration,Country.of.Origin
0,0,Arabica,conquista / morito,NaN,NaN,250.0,1.0,2012/2012,2012-01-13,Bourbon,...,10.0,7.00,78.33,0.11,0.0,0.0,Green,9.0,2013-01-12,2
1,1,Arabica,NaN,NaN,1775.0,275.0,70.0,None,2011-07-29,Caturra,...,10.0,8.17,83.08,0.01,0.0,0.0,NaN,0.0,2012-07-28,1
2,2,Arabica,la esmeralda,11/23/0634,4000.0,25.0,69.0,2017/2017,2017-09-08,Bourbon,...,10.0,7.50,82.58,0.10,0.0,1.0,Green,2.0,2018-09-08,2
3,3,Arabica,fazenda santo antonio,NaN,1000.0,305.0,2.0,2014/2014,2015-02-13,Catuai,...,10.0,7.58,83.00,0.00,0.0,0.0,Green,0.0,2016-02-13,0
4,4,Arabica,NaN,NaN,NaN,1.0,5.0,2013/2013,2014-09-12,NaN,...,10.0,7.25,82.08,0.11,1.0,0.0,Green,0.0,2015-09-12,1


# Data Preparation: Missing Values

In [13]:
# roadmap to correct columns missing values
col_type = {
    "Variety": "aroma, flavor, aftertaste, acidity, body, balance, uniformity, clean cup, sweetness, cupper points, total cup points, moisture",
    "Color": "variety",
    "Quakers": "variety, color, category one defect, category two defect",
    "Harvest.Year": "variety, gradient year, expiration",
    "Lot.Number": "farmname, harvest year, bag weight, number of bags, random",
    "Farm.Name": "lotnumber, random",
    "Altitude": "farm name, variety",
    "Processing.Method": "farm name, altitude, variety",
}

# Species                   0
# Farm.Name               166
# Lot.Number              480
# Altitude                 67
# Number.of.Bags            0
# Bag.Weight                0
# Harvest.Year             10
# Grading.Date              0
# Variety                  61
# Processing.Method        52
# Aroma                     0
# Flavor                    0
# Aftertaste                0
# Acidity                   0
# Body                      0
# Balance                   0
# Uniformity                0
# Clean.Cup                 0
# Sweetness                 0
# Cupper.Points             0
# Total.Cup.Points          0
# Moisture                  0
# Category.One.Defects      0
# Quakers                   1
# Color                   116
# Category.Two.Defects      0
# Expiration                0

In [14]:
# harvest year nan value

In [15]:
# farm name correction

# 1 - collect category of lot numbers that doesnt has a farm name
def collectLotNumber(df):
    lotnbr_category = {}
    for _, row in df[["Farm.Name", "Lot.Number"]].iterrows():
        if not pd.isna(row["Lot.Number"]) and pd.isna(row["Farm.Name"]):
            lotnbr_original = row["Lot.Number"]
            lotnbr = lotnbr_original.replace(" ", "")
            matches = [match.start() for match in re.finditer(f"[{string.punctuation}]", lotnbr)]
            special_caracter_index = None
            if len(matches) > 1:
                special_caracter_index = matches[1]
            elif len(matches) == 1:
                special_caracter_index = matches[0]
            if (special_caracter_index == None): 
                lotnbr_splitted = lotnbr
            else:
                lotnbr_splitted = lotnbr.split(lotnbr[special_caracter_index-1] + lotnbr[special_caracter_index])
            fazenda_name = "fazenda" + lotnbr_splitted[0]
            if fazenda_name in lotnbr_category:
                lotnbr_category[fazenda_name].append(lotnbr_original)
            else:
                lotnbr_category[fazenda_name] = [lotnbr_original]
    return lotnbr_category

# 2 - iter through the df rows where the lot number inside of the lotnbr_category is going to assign the col farm name to its value
def farmNameAssign(df):
    farmname_dict = collectLotNumber(df)
    for index, row in df[["Farm.Name", "Lot.Number"]].iterrows():
        for row2 in farmname_dict:
            if row["Lot.Number"] in farmname_dict[row2]:
                df.iloc[index, 2] = row2
    for i in train_df[train_df["Farm.Name"].isna()].index:
        train_df.iloc[i, 2] = "fazendo" + "".join(random.choice(string.ascii_letters + string.digits) for _ in range(5))
    return df

farmNameAssign(train_df)
farmNameAssign(test_df)

train_df["Farm.Name"].isna().sum()

0

In [16]:
# lot number correction
def lotNumberCorrection(df):
    for i in df[df["Lot.Number"].isna()].index:
        lotnbr_nan_farmname = df.iloc[i, 2]
        for j in df[df["Lot.Number"].notna()].index:
            lotnbr_farmname = df.iloc[j, 2]
            if (lotnbr_nan_farmname == lotnbr_farmname):
                df.iloc[i, 3] = df.iloc[j, 3]
                
    for i in df[df["Lot.Number"].isna()].index:
        df.iloc[i, 3] = "".join(random.choice(string.ascii_letters + string.digits) for _ in range (5))
    
lotNumberCorrection(train_df)
lotNumberCorrection(test_df)

train_df["Lot.Number"].isna().sum()

0

In [17]:
# altitude nan value
print(train_df["Altitude"].isna().sum())

for i in train_df[train_df["Altitude"].isna()].index:
    nan_altitude_farmname = train_df.iloc[i, 2]
    for j in train_df[train_df["Altitude"].notna()].index:
        altitude_farmname = train_df.iloc[j, 2]
        if nan_altitude_farmname == altitude_farmname:
            train_df.iloc[i, 4] = train_df.iloc[j, 4]
            
print(train_df["Altitude"].isna().sum())

68
42


In [18]:
train_df["Quakers"].isna().sum()

1

In [19]:
# quakers nan value (category of quakers: defects )
def category_of_quakers(df):
    cat_quakers = {}
    for _, row in df[["Category.One.Defects", "Category.Two.Defects", "Quakers"]].iterrows():
        q = row["Quakers"]
        cat_1 = row["Category.One.Defects"]
        cat_2 = row["Category.Two.Defects"]
        if q not in cat_quakers:
            cat_quakers[q] = {
                "Category.One.Defects": [cat_1],
                "Category.Two.Defects": [cat_2]
            }
        else:
            if cat_1 not in cat_quakers[q]["Category.One.Defects"]:
                cat_quakers[q]["Category.One.Defects"].append(cat_1)
            if cat_2 not in cat_quakers[q]["Category.Two.Defects"]:
                cat_quakers[q]["Category.Two.Defects"].append(cat_2)
    return cat_quakers

def convert_quakers_nan(df):
    df_without_nan = df[df["Quakers"].notna()]
    cat_quakers = category_of_quakers(df_without_nan)

    df_with_nan = df[df["Quakers"].isna()]
    id = pd.NA
    quakers = pd.NA
    
    for _, row in df_with_nan.iterrows():
        cat_1 = row["Category.One.Defects"]
        cat_2 = row["Category.Two.Defects"]
        for rank in cat_quakers:
            cat_1_rank = cat_quakers[rank]["Category.One.Defects"]
            cat_2_rank = cat_quakers[rank]["Category.Two.Defects"]
            if cat_1 in cat_1_rank and cat_2 in cat_2_rank:
                id = row["id"]
                quakers = rank
    
    if pd.notna(id) & pd.notna(quakers):
        for index, row in df.iterrows():
            if row["id"] == id:
                df.iloc[index, 24] = quakers
    
    return df


convert_quakers_nan(train_df)

,id,Species,Farm.Name,Lot.Number,Altitude,Number.of.Bags,Bag.Weight,Harvest.Year,Grading.Date,Variety,...,Sweetness,Cupper.Points,Total.Cup.Points,Moisture,Category.One.Defects,Quakers,Color,Category.Two.Defects,Expiration,Country.of.Origin
0,0,Arabica,conquista / morito,B2aVH,4600.0,250.0,1.0,2012/2012,2012-01-13,Bourbon,...,10.0,7.00,78.33,0.11,0.0,0.0,Green,9.0,2013-01-12,2
1,1,Arabica,fazendo8ro55,PifJY,1775.0,275.0,70.0,None,2011-07-29,Caturra,...,10.0,8.17,83.08,0.01,0.0,0.0,NaN,0.0,2012-07-28,1
2,2,Arabica,la esmeralda,11/23/0634,4000.0,25.0,69.0,2017/2017,2017-09-08,Bourbon,...,10.0,7.50,82.58,0.10,0.0,1.0,Green,2.0,2018-09-08,2
3,3,Arabica,fazenda santo antonio,uxG2n,1000.0,305.0,2.0,2014/2014,2015-02-13,Catuai,...,10.0,7.58,83.00,0.00,0.0,0.0,Green,0.0,2016-02-13,0
4,4,Arabica,fazendoeqvTm,dCsyq,NaN,1.0,5.0,2013/2013,2014-09-12,NaN,...,10.0,7.25,82.08,0.11,1.0,0.0,Green,0.0,2015-09-12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,580,Arabica,fazendoZhZz6,0MiYd,1800.0,275.0,2.0,2015/2016,2015-06-05,Other,...,10.0,7.83,84.25,0.12,0.0,0.0,Green,0.0,2016-06-04,1
581,581,Arabica,cafetal,101,1300.0,200.0,69.0,2017/2017,2017-07-03,Mundo Novo,...,10.0,7.83,83.92,0.11,0.0,0.0,Green,5.0,2018-07-03,3
582,582,Arabica,las lomas,eQE00,1200.0,250.0,1.0,2012/2012,2012-07-11,Bourbon,...,10.0,7.67,83.00,0.11,5.0,0.0,Green,8.0,2013-07-11,3
583,583,Arabica,fazendoUjwtR,VI7Zg,1800.0,250.0,70.0,2010/2010,2011-02-09,NaN,...,10.0,7.42,82.17,0.08,0.0,0.0,NaN,0.0,2012-02-09,1


In [20]:
train_df.isna().sum()

id                        0
Species                   0
Farm.Name                 0
Lot.Number                0
Altitude                 42
Number.of.Bags            0
Bag.Weight                0
Harvest.Year             17
Grading.Date              0
Variety                  61
Processing.Method        52
Aroma                     0
Flavor                    0
Aftertaste                0
Acidity                   0
Body                      0
Balance                   0
Uniformity                0
Clean.Cup                 0
Sweetness                 0
Cupper.Points             0
Total.Cup.Points          0
Moisture                  0
Category.One.Defects      0
Quakers                   0
Color                   116
Category.Two.Defects      0
Expiration                0
Country.of.Origin         0
dtype: int64

In [21]:
# variety nan value
# Aroma                     0
# Flavor                    0
# Aftertaste                0
# Acidity                   0
# Body                      0
# Balance                   0
# Uniformity                0
# Clean.Cup                 0
# Sweetness                 0
# Cupper.Points             0
# Total.Cup.Points          0
# Moisture                  0
# Category.One.Defects      0
# Quakers                   0
# Category.Two.Defects      0
# Expiration                0

def coffee_variety_categorization(filtered_df):
    coffee_variety_category = {}
    for index, row in filtered_df.iterrows():
        variety = row["Variety"]
        del row["Variety"]
        for name in row.to_dict():
            row[name] = [row[name]]
        if variety not in coffee_variety_category:
            coffee_variety_category[variety] = row.to_dict()
        else:
            for col in coffee_variety_category[variety]:
                coffee_variety_category[variety][col].append(row[col][0])
    return coffee_variety_category

def coffee_variety_nan_transformation(df):
    # dataframe without nan values and selected columns
    filtered_df = df[df[[
            "Variety" ,"Aroma", "Flavor", "Aftertaste", "Acidity", "Body", "Balance", 
            "Uniformity", "Clean.Cup", "Sweetness", "Cupper.Points", "Total.Cup.Points",
            "Moisture", "Category.One.Defects", "Quakers", "Category.Two.Defects", "Expiration"
        ]].notna()]
    selected_df = filtered_df[[
            "Variety" ,"Aroma", "Flavor", "Aftertaste", "Acidity", "Body", "Balance", 
            "Uniformity", "Clean.Cup", "Sweetness", "Cupper.Points", "Total.Cup.Points",
            "Moisture", "Category.One.Defects", "Quakers", "Category.Two.Defects", "Expiration"
        ]]
    # categorize existing varieties
    coffee_variety_category = coffee_variety_categorization(selected_df)
    # iter categories variety and classify the nan variety rows
    for index, row in df.iterrows():
        variety_target = row["Variety"]
        if pd.isna(variety_target):
            for coffee_variety in coffee_variety_category:
                for col in coffee_variety_category[coffee_variety]:
                    # check for the same caracteristics of the coffee bean in check
                    continue
            else:
                continue
        else:
            continue
    return coffee_variety_category

coffee_variety_nan_transformation(train_df)

{'Bourbon': {'Aroma': [7.17,
   7.5,
   7.83,
   7.5,
   7.5,
   8.0,
   7.58,
   7.83,
   7.17,
   7.0,
   7.0,
   7.5,
   7.17,
   7.75,
   7.42,
   7.58,
   7.33,
   7.58,
   7.33,
   7.83,
   7.58,
   6.5,
   7.58,
   8.0,
   8.33,
   7.42,
   7.75,
   7.83,
   7.83,
   7.5,
   7.67,
   7.42,
   7.5,
   7.75,
   7.92,
   6.75,
   7.67,
   7.75,
   7.92,
   7.5,
   8.42,
   7.67,
   7.5,
   7.67,
   7.5,
   7.25,
   8.0,
   7.5,
   7.58,
   8.0,
   7.67,
   7.58,
   7.5,
   6.17,
   7.5,
   7.5,
   7.58,
   7.67,
   7.83,
   7.75,
   7.67,
   7.58,
   7.67,
   7.67,
   7.0,
   7.67,
   7.25,
   7.5,
   7.5,
   7.67,
   7.58,
   7.5,
   6.75,
   8.0,
   7.92,
   8.5,
   7.75,
   7.5,
   7.33,
   7.42,
   7.58,
   7.5,
   7.33,
   7.5,
   7.42,
   7.5,
   7.58,
   7.67,
   7.67,
   7.42,
   7.42,
   7.33,
   7.92,
   7.08,
   7.75,
   7.67,
   7.67,
   7.42,
   7.5,
   7.67,
   7.58,
   7.67,
   7.42,
   7.58,
   7.67,
   7.17,
   7.67,
   7.5,
   7.67,
   7.5,
   7.58,
   7.67,
   7.

In [22]:
train_df[[
            "Variety" ,"Aroma", "Flavor", "Aftertaste", "Acidity", "Body", "Balance", 
            "Uniformity", "Clean.Cup", "Sweetness", "Cupper.Points", "Total.Cup.Points",
            "Moisture", "Category.One.Defects", "Quakers", "Category.Two.Defects", "Expiration"
        ]].notna()

,Variety,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean.Cup,Sweetness,Cupper.Points,Total.Cup.Points,Moisture,Category.One.Defects,Quakers,Category.Two.Defects,Expiration
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
581,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
582,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
583,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [23]:
train_df.columns

Index(['id', 'Species', 'Farm.Name', 'Lot.Number', 'Altitude',
       'Number.of.Bags', 'Bag.Weight', 'Harvest.Year', 'Grading.Date',
       'Variety', 'Processing.Method', 'Aroma', 'Flavor', 'Aftertaste',
       'Acidity', 'Body', 'Balance', 'Uniformity', 'Clean.Cup', 'Sweetness',
       'Cupper.Points', 'Total.Cup.Points', 'Moisture', 'Category.One.Defects',
       'Quakers', 'Color', 'Category.Two.Defects', 'Expiration',
       'Country.of.Origin'],
      dtype='object')

In [24]:
# altitude correction

# how to discover altitude? farmname-association
# categorize from...
# Species, Farm.Name, Lot.Number, Number.of.Bags, Bag.Weight, 
# Aroma, Flavor, Aftertaste, Acidity, Body, Balance, 
# Uniformity, Clean.Cup, Sweetness, Cupper.Points, Total.Cup.Points, Moisture, Category.One.Defects

# 1 - check the percent of altitude is associate to farmname, by infering the farmname nan value causing altitude nan value
# 2 - after know that altitude is strongly associate with farmname, we can fill altitude nan value by combinig farmname values + country origin

def altitude_farmname_association_percent(df):
    farmname_nan_count = df["Farm.Name"].isna().sum()
    altitude_nan_count = df["Altitude"].isna().sum()
    farmname_altitude_count = farmname_nan_count - (farmname_nan_count-altitude_nan_count)
    return (farmname_altitude_count/altitude_nan_count)*100

print(f"TRAIN DF, frequent of altitude nan value is associate with farmname nan value: {altitude_farmname_association_percent(train_df)}%")
print(f"TEST DF, frequent of altitude nan value is associate with farmname nan value: {altitude_farmname_association_percent(test_df)}%")

TRAIN DF, frequent of altitude nan value is associate with farmname nan value: 100.0%
TEST DF, frequent of altitude nan value is associate with farmname nan value: 100.0%


In [25]:
# harvest year correction

# how to discover harvest year? expiration-date
# categorize from...
# Species, Farm.Name, Lot.Number, Altitude, Number.of.Bags, Bag.Weight, Harvest.Year, Grading.Date, 
# Variety, Processing.Method, Aroma, Flavor, Aftertaste, Acidity, Body, Balance, 
# Uniformity, Clean.Cup, Sweetness, Cupper.Points, Total.Cup.Points, Moisture, Category.One.Defects, Quakers, 
# Color, Category.Two.Defects, Expiration, Country.of.Origin,
# species, variety, processing method

In [26]:
# variety correction

# how to discover variety? species-complex
# categorize from...
# Species, Farm.Name, Lot.Number, Altitude, Number.of.Bags, Bag.Weight, Harvest.Year, Grading.Date, 
# Variety, Processing.Method, Aroma, Flavor, Aftertaste, Acidity, Body, Balance, 
# Uniformity, Clean.Cup, Sweetness, Cupper.Points, Total.Cup.Points, Moisture, Category.One.Defects, Quakers, 
# Color, Category.Two.Defects, Expiration, Country.of.Origin,
# species, farm name, altitude, processing method, aroma, flavor, aftertaste, acidity, body, balance, uniformity, sweetness, clean cup

In [27]:
# processing method correction

# how to discover processing method? process-complex
# categorize from...
# Species, Farm.Name, Lot.Number, Altitude, Number.of.Bags, Bag.Weight, Harvest.Year, Grading.Date, 
# Variety, Processing.Method, Aroma, Flavor, Aftertaste, Acidity, Body, Balance, 
# Uniformity, Clean.Cup, Sweetness, Cupper.Points, Total.Cup.Points, Moisture, Category.One.Defects, Quakers, 
# Color, Category.Two.Defects, Expiration, Country.of.Origin,
# farm name, altitude, variety, category one and two defects

In [28]:
# quakers correction

# how to discover quakers?
# categorize from...
# Species, Farm.Name, Lot.Number, Altitude, Number.of.Bags, Bag.Weight, Harvest.Year, Grading.Date, 
# Variety, Processing.Method, Aroma, Flavor, Aftertaste, Acidity, Body, Balance, 
# Uniformity, Clean.Cup, Sweetness, Cupper.Points, Total.Cup.Points, Moisture, Category.One.Defects, Quakers, 
# Color, Category.Two.Defects, Expiration, Country.of.Origin,

In [29]:
# colors correction

# how to discover colors?
# categorize from...
# Species, Farm.Name, Lot.Number, Altitude, Number.of.Bags, Bag.Weight, Harvest.Year, Grading.Date, 
# Variety, Processing.Method, Aroma, Flavor, Aftertaste, Acidity, Body, Balance, 
# Uniformity, Clean.Cup, Sweetness, Cupper.Points, Total.Cup.Points, Moisture, Category.One.Defects, Quakers, 
# Color, Category.Two.Defects, Expiration, Country.of.Origin,

In [30]:
# convert all features nan values into insightful values
def convert_nan():
    return None

# Feature Engineering & Selection

# Model Selection & Training

# Model Evaluation & Validation

# Submit Model Predictions